In [1]:
import time
import numpy as np
import itertools
import math
from functools import reduce # python3 compatibility
from operator import mul
import json

In [2]:
# Functions for generating environment with capability
# generating task
def gen_tasks(task_num, max_capNum, capabilities): # n is the number of task, max_capNum is the maximum number of cap a task could require
    return [sorted(np.random.choice(capabilities, np.random.randint(1, max_capNum+1),replace=False)) for j in range(0, task_num)]


def gen_constraints(agent_num, task_num, power=1, a_min_edge=2,
                    t_max_edge=5):  # power is the inforce you put in the probabilities

    # the maximum tasks an agent could work on depends on the number of tasks available (e.g, if |T| = 1/2|A|, the roughly each agent can work on two tasks)
    # calculate the max and min edges for agents
    seats = math.floor(t_max_edge * task_num)
    a_taskInds = [[] for i in range(0, agent_num)]
    t_counter = [0 for j in range(0, task_num)]  # each indicate the current number of agents on the task

    ## generating the number of tasks the agents could work on.
    a_taskNums = []
    for i in range(0, agent_num):
        a_max_edge = min((seats - (agent_num - 1 - i) * a_min_edge), t_max_edge)
        a_min_edge = min(a_min_edge, a_max_edge)
        a_taskNums.append(
            np.random.randint(a_min_edge, a_max_edge + 1))  # indicate the number of task the agent could work on
        seats -= a_taskNums[i]

    t_indexes = [j for j in range(0, task_num) if
                 t_counter[j] < t_max_edge]  # make sure no further draw for those reached the maximum limit.
    for i in range(0, agent_num):
        if any([tc == 0 for tc in t_counter]):
            t_prob = [(math.e ** (t_max_edge - t_counter[j])) ** power for j in
                      t_indexes]  # power is used to manify the probability
            sum_prob = sum(t_prob)
            t_prop_2 = [prop / sum_prob for prop in t_prob]

            # draw tasks accounting to their current allocations
            a_taskInds[i] = list(np.random.choice(t_indexes, min(a_taskNums[i], len(t_indexes)), replace=False,
                                                  p=[prop / sum_prob for prop in t_prob]))
            # increase the chosen task counters
        else:
            a_taskInds[i] = list(np.random.choice(t_indexes, min(a_taskNums[i], len(t_indexes)), replace=False))

        for j in a_taskInds[i]:
            t_counter[j] += 1
        t_indexes = [j for j in range(0, task_num) if
                     t_counter[j] < t_max_edge]  # make sure no further draw for those reached the maximum limit.

    # get also the list of agents for each task
    t_agents = [[i for i in range(0, agent_num) if j in a_taskInds[i]] for j in range(0, task_num)]

    return a_taskInds, t_agents


def gen_agents(constraints, tasks, max_capNum, capabilities,
               max_capVal):  # m is the number of task, max_capNum is the maximum number of cap a task could require, max_capVal is the maximum capability value

    a_taskInds = constraints[0]
    agent_num = len(a_taskInds)
    caps_lists = []
    contri_lists = []
    for i in range(0, agent_num):
        t_caps = [tasks[j] for j in a_taskInds[i]]  # lists of caps that each task agent could perform

        caps_union = set(itertools.chain(*t_caps))  # union of caps of tasks that agent could perform
        a_cap_num = np.random.randint(min(3, max_capNum, len(caps_union)),
                                      min(len(caps_union), max_capNum) + 1)  # the num of caps the agent will have

        a_caps = set([np.random.choice(t_c) for t_c in
                      t_caps])  # initial draw to guarantee the agent has some contribution to each of the task that he could do

        rest_choices = list(caps_union.difference(a_caps))
        if rest_choices != []:
            # if min(len(rest_choices), a_cap_num - len(a_taskInds[i])) < 0:
            #     print("negative number a_cap_num - len(a_taskInds[i]): {}!".format(a_cap_num - len(a_taskInds[i])))
            # else:
            update_len = max(0, a_cap_num - len(a_taskInds[i]))
            a_caps.update(np.random.choice(rest_choices, min(len(rest_choices), update_len), replace=False))

        caps_lists.append(sorted(list(a_caps)))

        contri_lists.append(
            [(np.random.randint(1, max_capVal + 1) if c in caps_lists[i] else 0) for c in range(0, len(capabilities))])
    return caps_lists, contri_lists



In [3]:
def random(agents, tasks, constraints, gamma = 1):
    task_num = len(tasks)
    agent_num = len(agents)
    a_taskInds = constraints[0]
    alloc = [np.random.choice(a_taskInds[i]+[task_num]) for i in range(0,agent_num)]
    return alloc, sys_reward_agents(agents,tasks, alloc, gamma)

In [4]:
def task_reward(task, agents, gamma=1):  # task is represented by a list of capabilities it requires, agents is
    # a list of agents, where each represented by a list cap contribution values
    if agents == []:
        return 0
    else:
        return sum([max([agent[cap] for agent in agents]) for cap in task]) * (gamma ** len(agents))


def sys_reward_agents(agents, tasks, alloc, gamma=1): #alloc is a vector of size M each element indicate which task the agent is alloated to
    return sum([task_reward(tasks[j], [agents[i] for i in range(0,len(agents)) if alloc[i] == j], gamma) for j in range(0, len(tasks))])


def resultCal(agents, tasks, constraints, r_msgs, iteration, iter_over, converge, gamma=1):
    a_taskInds = constraints[0]
    agent_num = len(agents)
    task_num = len(tasks)
    a_msg_sum = [{d_key: sum([r_msgs[j][i][0] for j in a_taskInds[i] if j != d_key])
                         + r_msgs[d_key][i][1] for d_key in a_taskInds[i]}
                 for i in range(0, agent_num)]
    alloc = [max(ams, key=ams.get) if ams != {} else task_num for ams in a_msg_sum]
    return alloc, sys_reward_agents(agents, tasks, alloc, gamma), iteration, iter_over, converge


# Agent contribution
def agent_con(competency_lists, t_caps_lists, query_agentIndex, query_taskIndex, member, a_taskInds):
    # the marginal contribution of an agent to a coalition
    if query_taskIndex == len(t_caps_lists):
        return 0
    if query_taskIndex not in a_taskInds[query_agentIndex]:
        return 0
    curr_pro = [competency_lists[mem] for mem in member]
    cur_reward = task_reward(t_caps_lists[query_taskIndex], curr_pro, gamma=1)
    if query_agentIndex in member:
        remained_mems = member[:]
        remained_mems.remove(query_agentIndex)  # the remained coalition members besides of agent i
        coal_pro = [competency_lists[mem] for mem in remained_mems]
        new_reward = task_reward(t_caps_lists[query_taskIndex], coal_pro, gamma=1)
        return cur_reward - new_reward
    else:
        added_mems = member[:]
        added_mems.append(query_agentIndex)
        coal_pro = [competency_lists[mem] for mem in added_mems]
        new_reward = task_reward(t_caps_lists[query_taskIndex], coal_pro, gamma=1)
        return new_reward - cur_reward


def aim_tasks(movement_values, a_tasks, a_index):
    max_mv = max(movement_values['agent ' + str(a_index)])
    target_tasks = []  # aiming tasks are the tasks the agent makes proposal to
    max_num = movement_values['agent ' + str(a_index)].count(max_mv)
    first_pos = 0
    for ind in range(max_num):
        new_list = movement_values['agent ' + str(a_index)][first_pos:]
        index_add_t = first_pos + new_list.index(max_mv)
        act_task = a_tasks[a_index][index_add_t]
        target_tasks.append(act_task)
        next_pos = new_list.index(max_mv) + 1
        first_pos += next_pos
    return target_tasks


def accepted_tasks(instruction_received, propose_states, a_index):
    accept_num = instruction_received['agent ' + str(a_index)].count(1)
    accept_tasks = []  # the index of accept message, which corresponding to the index of sending tasks
    first_pos = 0
    for ind in range(accept_num):
        new_list = instruction_received['agent ' + str(a_index)][first_pos:]
        new_t_ind = first_pos + new_list.index(1)
        act_task = propose_states['agent ' + str(a_index)][new_t_ind]
        accept_tasks.append(act_task)
        next_pos = new_list.index(1) + 1
        first_pos += next_pos
    return accept_tasks



In [5]:
def OPD(agents, tasks, constraints, gamma):
    task_num = len(tasks)
    agent_num = len(agents)
    a_taskInds = [list(con) for con in constraints[0]]
    t_agentInds = [list(con) for con in constraints[1]]

    a_ubs = [[0 for j in a_taskInds[i]] for i in range(0, agent_num)]
    a_lbs = [[0 for j in a_taskInds[i]] for i in range(0, agent_num)]

    for j in range(0, task_num):

        linked_agentInds = t_agentInds[j]
        com_dict = []
        com_rewards = []
        for c in itertools.product(*[[0, 1] for i in linked_agentInds]):
            com_dict.append({linked_agentInds[i]: c[i] for i in range(0, len(c))})
            com_rewards.append(
                task_reward(tasks[j], [agents[a_key] for a_key in com_dict[-1].keys() if com_dict[-1][a_key] == 1],
                            gamma)
            )

        for i in t_agentInds[j]:
            t_ind = a_taskInds[i].index(j)
            cons_j = [com_rewards[c]
                      for c in range(0, len(com_dict)) if com_dict[c][i] == 1]

            a_lbs[i][t_ind] = min(cons_j)
            a_ubs[i][t_ind] = max(cons_j)

    for i in range(0, agent_num):
        t_flag = [True for j in a_taskInds[i]]
        for t_ind in range(0, len(a_taskInds[i])):
            for t2_ind in range(0, len(a_taskInds[i])):
                if t_ind != t2_ind and a_ubs[i][t_ind] < a_lbs[i][t2_ind]:
                    t_flag[t_ind] = False
                    break

        for t_ind in range(0, len(t_flag)):
            if not t_flag[t_ind]:
                t_agentInds[a_taskInds[i][t_ind]].remove(i)

        new_a_taskInds = [a_taskInds[i][t_ind]
                          for t_ind in range(0, len(a_taskInds[i])) if t_flag[t_ind]]
        a_taskInds[i] = new_a_taskInds

    return a_taskInds, t_agentInds


def FMS(agents, tasks, constraints, gamma, time_bound):
    converge = False
    iter_over = False
    record_u = []
    record_t = []
    start_time = time.time()
    a_taskInds = constraints[0]
    t_agentInds = constraints[1]
    task_num = len(tasks)
    agent_num = len(agents)

    q_msgs = [{t_key: {} for t_key in a_taskInds[i]} for i in range(0, agent_num)]
    r_msgs = [
        {t_agentInds[j][i]: ({1: -100} if len(a_taskInds[t_agentInds[j][i]]) == 1 else {key: -100 for key in [0, 1]})
         for i in range(0, len(t_agentInds[j]))}
        for j in range(0, task_num)]

    q_flags = [False for i in range(0, agent_num)]
    r_flags = [False for j in range(0, task_num)]

    iteration = 0
    while iteration < agent_num + task_num:  # True:
        if time.time() - start_time >= time_bound:
            r = resultCal(agents, tasks, constraints, r_msgs, iteration, iter_over, converge, gamma)
        iteration += 1

        if iteration > agent_num + task_num:
            iter_over = True
            r = resultCal(agents, tasks, constraints, r_msgs, iteration, iter_over, converge, gamma)

        if all(q_flags) and all(r_flags):  # converge, msgs are all the same.
            converge = True
            # break
        for i in range(0, agent_num):
            linked_taskInds = a_taskInds[i]

            flag = True
            for t_key in linked_taskInds:

                ####### check time bound
                if time.time() - start_time >= time_bound:
                    r = resultCal(agents, tasks, constraints, r_msgs, iteration, iter_over, converge, gamma)
                ####### check time bound
                msgs = {}

                if len(linked_taskInds) > 1:
                    msgs[1] = sum([m[0] for m in [r_msgs[j][i] for j in linked_taskInds if j != t_key]])
                    msg_0 = []
                    ts = list(linked_taskInds)
                    ts.remove(t_key)

                    for k in ts:
                        msg_0.append(sum([m[0] for m in [r_msgs[j][i] for j in ts if j != k]])
                                     + r_msgs[k][i][1])

                    msgs[0] = (0 if msg_0 == [] else max(msg_0))
                else:
                    msgs[1] = 0

                alphas = -sum(msgs.values()) / len(msgs.keys())

                msgs_regularised = {d_key: msgs[d_key] + alphas for d_key in msgs.keys()}

                old_msg = q_msgs[i][t_key]
                if old_msg != {} and any([abs(msgs_regularised[d_key] - old_msg[d_key]) > 10 ** (-5)
                                          for d_key in old_msg.keys()]):
                    flag = False

                q_msgs[i][t_key] = msgs_regularised

            if flag:  # agent i sending the same info
                q_flags[i] = True

        if time.time() - start_time >= time_bound:
            break
        ###################### SAME thing, using comprehension
        #             msgs = {t_key:{d_key:sum([m[d_key] for m in [r_msgs[j][i] for j in linked_taskInds if j != t_key]])
        #                            for d_key in linked_taskInds}
        #                                 for t_key in linked_taskInds}
        #             alphas = {t_key:-sum(msgs[t_key].values())/len(msgs.keys())
        #                       for t_key in linked_taskInds}
        #             msgs_regularised = {t_key:{d_key:msgs[t_key][d_key] + alphas[t_key]
        #                            for d_key in linked_taskInds}
        #                                 for t_key in linked_taskInds}
        for j in range(0, task_num):
            linked_agentInds = t_agentInds[j]
            # msg_con = [q_msgs[a][j] for a in linked_agentInds]

            com_dict = []
            com_rewards = []
            dom_com = [[0, 1] if len(a_taskInds[i]) > 1 else [1] for i in linked_agentInds]

            for c in itertools.product(*dom_com):
                ####### check time bound
                if time.time() - start_time >= time_bound:
                    r = resultCal(agents, tasks, constraints, r_msgs, iteration, iter_over, converge, gamma)
                ####### check time bound

                com_dict.append({linked_agentInds[i]: c[i] for i in range(0, len(c))})
                com_rewards.append(
                    task_reward(tasks[j], [agents[a_key] for a_key in com_dict[-1].keys() if com_dict[-1][a_key] == 1],
                                gamma)
                )

            flag = True
            for a_key in linked_agentInds:

                ####### check time bound
                if time.time() - start_time >= time_bound:
                    r = resultCal(agents, tasks, constraints, r_msgs, iteration, iter_over, converge, gamma)
                ####### check time bound

                old_msg = r_msgs[j][a_key]
                q_table = []
                for c in range(0, len(com_dict)):
                    q_table.append(sum([q_msgs[a][j][com_dict[c][a]] for a in linked_agentInds if a != a_key])
                                   + com_rewards[c])

                r_msgs[j][a_key] = {
                    d_key: max([q_table[c] for c in range(0, len(com_dict)) if com_dict[c][a_key] == d_key])
                    for d_key in ([0, 1] if len(a_taskInds[a_key]) > 1 else [1])}

                if any([abs(r_msgs[j][a_key][d_key] - old_msg[d_key]) > 10 ** (-5) for d_key in old_msg.keys()]):
                    flag = False

            if flag:  # task j sending the same info
                r_flags[j] = True

        record_t.append(time.time() - start_time)
        r = resultCal(agents, tasks, constraints, r_msgs, iteration, iter_over, converge, gamma)        
        record_u.append(r[1])

    return record_u, record_t   


In [6]:
def embed_DSA(constraints, tasks, agents, probability, time_bound):
    converge = False
    record_u = []
    record_t = []
    record_message = []
    agent_num = len(agents)
    task_num = len(tasks)
    a_tasks = constraints[0]
    t_agents = constraints[1]
    contributions = {}
    for j in range(0, task_num):
        contributions['towards task ' + str(j)] = list(np.zeros(len(t_agents[j]), int))
    received_contribution = {} # zero at the task_num-th is the contribution of an agent assigned to no tasks
    for i in range(0, agent_num):
        received_contribution['agent ' + str(i)] = list(np.zeros(len(a_tasks[i])+1, int))
    movement_values = {}
    CS = [[] for j in range(0, task_num+1)]
    state = [task_num for i in range(0, agent_num)]
    update_t = range(0, task_num)
    iter = 0
    
    is_continue = True
    start_time = time.time()
    while is_continue:  # > 0:
        message_pass_num = 0
        if time.time() - start_time >= time_bound:
            break
        iter += 1

        # update agents' allocation state
        for j in range(0, task_num):
            member_agents = CS[j]
            for i in member_agents:
                state[i] = int(j)

        # tasks calculate agents contribution to them
        for j in range(0, task_num):
            if j in update_t:
                for i in t_agents[j]:
                    message_pass_num += 1
                    current_coal = CS[j]
                    if len(current_coal) == 0:
                        coal = [agents[i]]
                        add_reward = task_reward(tasks[j], coal, gamma=1)
                        ini_contrib = add_reward
                    else:
                        coal = [agents[mem] for mem in current_coal]
                        cur_reward = task_reward(tasks[j], coal, gamma=1)
                        if i in current_coal:
                            remained_mems = current_coal[:]
                            remained_mems.remove(i)
                            if len(remained_mems) == 0:
                                non_a_reward = 0
                            else:
                                remain_coal = [agents[remai_mem] for remai_mem in remained_mems]
                                non_a_reward = task_reward(tasks[j], remain_coal, gamma=1)
                            ini_contrib = cur_reward - non_a_reward
                        else:
                            coal.append(agents[i])
                            add_reward = task_reward(tasks[j], coal, gamma=1)
                            ini_contrib = add_reward - cur_reward
                    j_index = a_tasks[i].index(j)
                    received_contribution['agent ' + str(i)][j_index] = ini_contrib
                    i_ind = t_agents[j].index(i)
                    contributions['towards task ' + str(j)][i_ind] = ini_contrib

        # agents calculate their movement values #
        for i in range(0, agent_num):
            current_task = state[i]   # the agent's current task
            if current_task == task_num:
                current_contribution = 0
            else:
                cur_t_ind = a_tasks[i].index(current_task)
                current_contribution = received_contribution['agent ' + str(i)][cur_t_ind]
            movement_values['agent ' + str(i)] = [received_contribution['agent ' + str(i)][mess_ind]-current_contribution for mess_ind in range(0, len(received_contribution['agent ' + str(i)]))]

        # agents choose their tasks  ##########
        update_t = []
        max_mv = [[] for i in range(0, agent_num)]
        for i in range(0, agent_num):
            max_mv[i] = max(movement_values['agent ' + str(i)])
            if max_mv[i] > 0:
                pro = np.random.random(1)
                if pro > 1 - probability:
                    old_task = state[i]
                    potential_task = []
                    max_num = movement_values['agent ' + str(i)].count(max_mv[i])
                    first_pos = 0
                    for ind in range(max_num):
                        new_list = movement_values['agent ' + str(i)][first_pos:]
                        potential_task.append(first_pos + new_list.index(max_mv[i]))
                        next_pos = new_list.index(max_mv[i]) + 1
                        first_pos += next_pos
                    choose_t_ind = np.random.randint(len(potential_task))
                    choosed_task = a_tasks[i][potential_task[choose_t_ind]]
                    # message_pass_num += 1
                    CS[choosed_task].append(i)
                    update_t.append(choosed_task)
                    state[i] = choosed_task
                    if old_task != task_num:
                        # message_pass_num += 1
                        CS[old_task].remove(i)
                        update_t.append(old_task)
        #  if continue
        if max(max_mv) > 0:
            is_continue = True
        else:
            is_continue = False
            converge = True
        
        record_t.append(time.time()- start_time)
        #  current system reward    #
        ini_task_u = [[] for i in range(0, task_num)]
        for j in range(0, task_num):
            if len(CS[j]) == 0:
                ini_task_u[j] = 0
            else:
                t_coalition = [agents[c_mem] for c_mem in CS[j]]
                ini_task_u[j] = task_reward(tasks[j], t_coalition, gamma=1)
        global_u = sum(ini_task_u)
        record_u.append(global_u)
        record_message.append(message_pass_num)

    return record_u, record_t, record_message


In [7]:
def embed_disNE(constraints, tasks, agents, time_bound):

    converge = False
    agent_num = len(agents)
    task_num = len(tasks)
    a_tasks = constraints[0]
    t_agents = constraints[1]

    CS = [[] for j in range(0, task_num + 1)]
    contributions = {}
    for j in range(0, task_num):
        contributions['towards task ' + str(j)] = list(np.zeros(len(t_agents[j]), int))
    received_contribution = {}  # zero at the task_num-th is the contribution of an agent assigned to no tasks
    for i in range(0, agent_num):
        received_contribution['agent ' + str(i)] = list(np.zeros(len(a_tasks[i])+1, int))
    movement_values = {}

    state = [task_num for i in range(0, agent_num)]  # agents' current assignments, initially are no-task (task_num)
    iter = 0
    record_u = []
    record_t = []
    record_message = []
    update_t = range(0, task_num)
    is_continue = True
    start_time = time.time()
    while is_continue:  # > 0:
        message_pass_num = 0
        if time.time() - start_time >= time_bound:
            break
        iter += 1
        # update agents' allocation state
        for j in update_t:
            member_agents = CS[j]
            for i in member_agents:
                state[i] = int(j)

        # tasks calculate agents contribution to them
        for j in range(0, task_num):
            if j in update_t:
                for i in t_agents[j]:
                    message_pass_num += 1
                    current_coal = CS[j]
                    if len(current_coal) == 0:
                        coal = [agents[i]]
                        add_reward = task_reward(tasks[j], coal, gamma=1)
                        ini_contrib = add_reward
                    else:
                        coal = [agents[mem] for mem in current_coal]
                        cur_reward = task_reward(tasks[j], coal, gamma=1)
                        if i in current_coal:
                            remained_mems = current_coal[:]
                            remained_mems.remove(i)
                            if len(remained_mems) == 0:
                                non_a_reward = 0
                            else:
                                remain_coal = [agents[remai_mem] for remai_mem in remained_mems]
                                non_a_reward = task_reward(tasks[j], remain_coal, gamma=1)
                            ini_contrib = cur_reward - non_a_reward
                        else:
                            coal.append(agents[i])
                            add_reward = task_reward(tasks[j], coal, gamma=1)
                            ini_contrib = add_reward - cur_reward
                    j_index = a_tasks[i].index(j)
                    received_contribution['agent ' + str(i)][j_index] = ini_contrib
                    i_ind = t_agents[j].index(i)
                    contributions['towards task ' + str(j)][i_ind] = ini_contrib

        # agents calculate their movement values #
        for i in range(0, agent_num):
            current_task = state[i]   # the agent's current task
            if current_task == task_num:
                current_contribution = 0
            else:
                cur_t_ind = a_tasks[i].index(current_task)
                current_contribution = received_contribution['agent ' + str(i)][cur_t_ind]
            movement_values['agent ' + str(i)] = [received_contribution['agent ' + str(i)][mess_ind]-current_contribution for mess_ind in range(0, len(received_contribution['agent ' + str(i)]))]

        #   agents propose to tasks #
        received_proposal = [{'from agents': [], 'proposal values': []} for j in range(0, task_num)]   # the proposals received by tasks
        propose_states = {}    # which agents propose to which tasks
        propose_agents = []     # which agents make proposals
        received_tasks = []  # which tasks receive proposals
        max_mv = [[] for i in range(0, agent_num)]
        for i in range(0, agent_num):
            max_mv[i] = max(movement_values['agent ' + str(i)])
            if max_mv[i] > 0:
                propose_agents.append(i)
                aim_task = []   # aiming tasks are the tasks the agent makes proposal to
                max_num = movement_values['agent ' + str(i)].count(max_mv[i])
                first_pos = 0
                for ind in range(max_num):
                    new_list = movement_values['agent ' + str(i)][first_pos:]
                    index_add_t = first_pos + new_list.index(max_mv[i])
                    act_task = a_tasks[i][index_add_t]
                    aim_task.append(act_task)
                    next_pos = new_list.index(max_mv[i]) + 1
                    first_pos += next_pos
                old_task = state[i]
                if old_task != task_num:
                    aim_task.append(old_task)

                message_pass_num += len(aim_task)
                propose_states['agent ' + str(i)] = aim_task
                received_tasks.append(aim_task)
                for j in aim_task:
                    received_proposal[j]['from agents'].append(i)
                    received_proposal[j]['proposal values'].append(max_mv[i])

        # check whether or not to continue  #
        if max(max_mv) > 0:
            is_continue = True
        else:
            is_continue = False
            converge = True

        # tasks send instruction messages to agents #
        if is_continue:
            #  find out which coalitions received proposal
            active_tasks = []
            for j in received_tasks:
                active_tasks.extend(j)
            active_tasks = list(set(active_tasks))

            instruction_received = {}
            for i in propose_agents:
                instruction_received['agent ' + str(i)] = list(np.zeros(len(propose_states['agent ' + str(i)]), int))
                message_pass_num += len(propose_states['agent ' + str(i)])

            for j in active_tasks:
                if j == task_num:
                    for a_i in received_proposal[j]['from agents']:
                        a_i_index = propose_states['agent ' + str(int(a_i))].index(j)
                        instruction_received['agent ' + str(int(a_i))][a_i_index] = 1
                else:
                    max_pro = max(received_proposal[j]['proposal values'])
                    poent_accep_agent = []   # the agents who give the maximum proposal
                    poten_num = received_proposal[j]['proposal values'].count(max_pro)
                    first_pos = 0
                    for ind in range(poten_num):
                        new_list = received_proposal[j]['proposal values'][first_pos:]
                        index_add_a = first_pos + new_list.index(max_pro)
                        act_agent = received_proposal[j]['from agents'][index_add_a]
                        poent_accep_agent.append(act_agent)
                        next_pos = new_list.index(max_pro) + 1
                        first_pos += next_pos
                    choosed_agent = np.random.choice(poent_accep_agent)  # the agent index accepted by task j
                    acc_t_ind = propose_states['agent '+ str(int(choosed_agent))].index(j)   # the index of task j
                    instruction_received['agent '+str(int(choosed_agent))][acc_t_ind] = 1

        # agents move to tasks
        update_t = []
        for i in propose_agents:
            accept_num = instruction_received['agent '+str(i)].count(1)
            if accept_num == 0:
                continue
            else:
                first_pos = 0
                accept_tasks = []  # the index of accept message, which corresponding to the index of sending tasks
                for ind in range(accept_num):
                    new_list = instruction_received['agent ' + str(i)][first_pos:]
                    new_t_ind = first_pos + new_list.index(1)
                    act_task = propose_states['agent ' + str(i)][new_t_ind]
                    accept_tasks.append(act_task)
                    next_pos = new_list.index(1) + 1
                    first_pos += next_pos

                old_coa = state[i]
                if old_coa != task_num:
                    old_coa_ind = propose_states['agent '+ str(i)].index(old_coa)
                    if instruction_received['agent '+str(i)][old_coa_ind] == 0:  # agent's own task reject its movement
                        continue
                    else:
                        message_pass_num += 1
                        accept_tasks.remove(old_coa)  # the other accept-coalitions besides its own coalition
                if len(accept_tasks) == 0:
                    continue
                else:
                    message_pass_num += 1
                    choosed_task = np.random.choice(accept_tasks)  # random select a coalition that gives accept message
                CS[choosed_task].append(i)
                update_t.append(choosed_task)
                if old_coa != task_num:
                    de_i_ind = CS[old_coa].index(i)
                    del CS[old_coa][de_i_ind]
                    update_t.append(old_coa)
        
        record_t.append(time.time()-start_time)
        # current system reward #
        ini_task_u = [[] for j in range(0, task_num)]
        for j in range(0, task_num):
            if len(CS[j]) == 0:
                ini_task_u[j] = 0
            else:
                t_coalition = [agents[c_mem] for c_mem in CS[j]]
                ini_task_u[j] = task_reward(tasks[j], t_coalition, gamma=1)
        global_u = sum(ini_task_u)
        record_u.append(global_u)
        record_message.append(message_pass_num)

    return record_u, record_t, record_message

In [8]:
def append_record(record, filename, typ):
    with open(filename, 'a') as f:
        if typ != '':
            json.dump(record, f, default = typ)
        else:
            json.dump(record, f)
        f.write('\n')
        # f.write(os.linesep)
        f.close()

In [11]:
run_num = 1
probability = 0.7
time_bound = 300
capNum = 10
max_capNum_task = 10
max_capNum_agent = 10
max_capVal = 10
capabilities = list(range(0, capNum))
a_min_edge = 1
task_num = 100
agent_num = 2 * task_num

# t_max_edge = 6
# min_t_num = 2
# max_t_num = 2
# agent_num = 4
# agents = np.array([[0, 0, 0, 7, 6], [4, 3, 8, 8, 4], [0, 9, 8, 0, 0], [4, 9, 0, 7, 0]])
# tasks = np.array([[0, 1, 2], [0, 3, 4]])
# constraints = ([[0, 1], [0, 1], [0, 1], [0, 1]], [[0, 1, 2, 3], [0, 1, 2, 3]])


t_max_edge = 0.04 * task_num
tasks = gen_tasks(task_num, max_capNum_task, capabilities)
constraints = gen_constraints(agent_num, task_num, 1, a_min_edge, t_max_edge)
## caps_lists: capabilities the agent can provide; agents: the value of the agent provide the capabilities
caps_lists, agents = gen_agents(constraints, tasks, max_capNum_agent, capabilities, max_capVal)

# a_tasks = constraints[0]
# t_agents = constraints[1]
# for i in range(0, agent_num):
#     a_tasks[i].append(task_num)
# t_agents.append(list(range(0, agent_num)))

result = {"task_num": task_num, "agent_num": agent_num}


# r = random(agents, tasks,constraints, gamma=1 )
# alloc = r[0]
# result['rand'] = r[1]
# print("task_number: ", task_num, 'rand result',result['rand'])

start = time.time()
dsa_u, dsa_t, dsa_msg_num = embed_DSA(constraints, tasks, agents, probability, time_bound)
end = time.time()
result['dsa_u'] = dsa_u
result['dsa_t'] = dsa_t
result['dsa_msg'] = dsa_msg_num
print("DSA time:", end-start, dsa_u[-1])


start = time.time()
disNE_u, disNE_t, disNE_msg_num = embed_disNE(constraints, tasks, agents, time_bound)
end = time.time()
result['disNE_u'] = disNE_u
result['disNE_t'] = disNE_t
result['DisNE_msg'] = disNE_msg_num
print("disNE time:", end-start, disNE_u[-1])


start = time.time()
new_con = OPD(agents, tasks, constraints, gamma=1)
BnBFMS_u, BnBFMS_t = FMS(agents, tasks, new_con, gamma=1, time_bound=time_bound)
end = time.time()
result['BnBFMS_u'] = BnBFMS_u
result['BnBFMS_t'] = BnBFMS_t
print("BnBFMS time:", end-start, BnBFMS_u[-1])


# #         append data and result
# files = {'cap_mono2': [result, '']} #
# for filename in list(files.keys()):
#     append_record(files[filename][0], filename, typ=files[filename][1])






DSA time: 0.28623461723327637 7143
disNE time: 0.12566113471984863 7174
BnBFMS time: 435.6629014015198 7094
